In [73]:
import pandas as pd
import requests
from bs4 import BeautifulSoup 
import time

In [7]:
### We will scrape 4 web sites to get the qualifying performances for men and women for the trials
site_list = ["men-s-marathon-performances",
             "men-s-half-marathon-performances",
             "women-s-marathon-performances",
             "women-s-half-marathon-performances"]

In [8]:
df_dict = {}
for elem in site_list:
    df_dict[elem] = None

In [11]:
for site in site_list:
    url = f"https://www.usatf.org/events/2024/2024-u-s-olympic-team-trials-%e2%80%94-marathon/2024-u-s-olympic-team-trials-marathon-qualificatio/{site}" 
    r = requests.get(url) 
    soup = BeautifulSoup(r.content, 'html5lib') 
    table = soup.find_all('table')
    df = pd.read_html(str(table), header=0)[0]
    df['Category'] = site.split('-')[0]
    df['Distance'] = site.split('-')[2]
    df_dict[site] = df

In [14]:
df = pd.concat(df_dict.values()).reset_index(drop=True)
df.head()

,Name,Time,Race,Location,Date,Category,Distance
0,Conner Mantz,2:07:47,Chicago Marathon,"Chicago, IL",10/8/2023,men,marathon
1,Clayton Young,2:08:00,Chicago Marathon,"Chicago, IL",10/8/2023,men,marathon
2,Galen Rupp,2:08:48,Chicago Marathon,"Chicago, IL",10/8/2023,men,marathon
3,Sam Chelanga,2:08:50,Chicago Marathon,"Chicago, IL",10/8/2023,men,marathon
4,Scott Fauble,2:08:52,Boston Marathon,"Boston, MA",4/18/2022,men,marathon


In [15]:
df.tail()

,Name,Time,Race,Location,Date,Category,Distance
395,Jessa Hanson,1:11:24,USATF Half Marathon Championships,"Fort Worth, TX",2/26/2023,women,half
396,Jaci Smith,1:11:27,Garry Bjorklund Half Marathon,"Duluth, MN",6/17/2023,women,half
397,Grace Moore,1:11:53,Project 13.1,"Valley Cottage, NY",3/25/2023,women,half
398,Katrina Spratford,1:11:54,Mesa Half Marathon,"Mesa, AZ",2/4/2023,women,half
399,Rosa Moriello,1:11:58,Eugene Half Marathon,"Eugene, OR",4/30/2023,women,half


In [16]:
df.shape

(400, 7)

In [155]:
men_bios = "https://www.orlando2024trials.com/index.php?mact=MAMSDirectory,cntnt01,default,0&cntnt01group=Athlete&cntnt01prop=category&cntnt01propval=men&cntnt01returnid=34&cntnt01pagelimit=1000&cntnt01sortby=f%3Aqt&cntnt01sortorder=ASC&dir=ASC&filter=full" 
r = requests.get(men_bios) 
soup = BeautifulSoup(r.content, 'html5lib') 

In [157]:
links = soup.find_all(class_='stretched-link')
links = [link['href'] for link in links]
links

['https://www.orlando2024trials.com/athlete/130/34/conner-mantz',
 'https://www.orlando2024trials.com/athlete/370/34/clayton-young',
 'https://www.orlando2024trials.com/athlete/265/34/galen-rupp',
 'https://www.orlando2024trials.com/athlete/141/34/sam-chelanga',
 'https://www.orlando2024trials.com/athlete/376/34/scott-fauble',
 'https://www.orlando2024trials.com/athlete/125/34/elkanah-kibet',
 'https://www.orlando2024trials.com/athlete/161/34/zachery-panning',
 'https://www.orlando2024trials.com/athlete/75/34/leonard-korir',
 'https://www.orlando2024trials.com/athlete/375/34/futsum-zienasellassie',
 'https://www.orlando2024trials.com/athlete/384/34/brian-shrader',
 'https://www.orlando2024trials.com/athlete/159/34/matthew-alexander-mcdonald',
 'https://www.orlando2024trials.com/athlete/285/34/nicolas-rodolfo-montanez',
 'https://www.orlando2024trials.com/athlete/401/34/teshome-mekonen',
 'https://www.orlando2024trials.com/athlete/21/34/clayton-albertson',
 'https://www.orlando2024trial

In [158]:
women_bios = "https://www.orlando2024trials.com/index.php?mact=MAMSDirectory,cntnt01,default,0&cntnt01group=Athlete&cntnt01prop=category&cntnt01propval=women&cntnt01returnid=34&cntnt01pagelimit=1000&cntnt01sortby=f%3Aqt&cntnt01sortorder=ASC&dir=ASC&filter=full" 
r = requests.get(women_bios) 
soup = BeautifulSoup(r.content, 'html5lib') 
links += [link['href'] for link in soup.find_all(class_='stretched-link')]

In [159]:
links

['https://www.orlando2024trials.com/athlete/130/34/conner-mantz',
 'https://www.orlando2024trials.com/athlete/370/34/clayton-young',
 'https://www.orlando2024trials.com/athlete/265/34/galen-rupp',
 'https://www.orlando2024trials.com/athlete/141/34/sam-chelanga',
 'https://www.orlando2024trials.com/athlete/376/34/scott-fauble',
 'https://www.orlando2024trials.com/athlete/125/34/elkanah-kibet',
 'https://www.orlando2024trials.com/athlete/161/34/zachery-panning',
 'https://www.orlando2024trials.com/athlete/75/34/leonard-korir',
 'https://www.orlando2024trials.com/athlete/375/34/futsum-zienasellassie',
 'https://www.orlando2024trials.com/athlete/384/34/brian-shrader',
 'https://www.orlando2024trials.com/athlete/159/34/matthew-alexander-mcdonald',
 'https://www.orlando2024trials.com/athlete/285/34/nicolas-rodolfo-montanez',
 'https://www.orlando2024trials.com/athlete/401/34/teshome-mekonen',
 'https://www.orlando2024trials.com/athlete/21/34/clayton-albertson',
 'https://www.orlando2024trial

In [160]:
len(links)

306

In [161]:
profile_df_dict = {}

In [162]:
for i in range(len(links)):
    bio = links[i]
    r = requests.get(bio) 
    soup = BeautifulSoup(r.content, 'html5lib')
    profile = soup.find_all(class_="profiles")[0]
    profile = profile.text.split('\n    ')
    profile = [elem for elem in profile if elem != '']
    #profile = [elem.strip() for elem in profile]
    profile_keys = profile[0::2]
    profile_values = profile[1::2]
    profile_keys = [elem for elem in profile_keys if len(elem) > 0 and elem != '\n\n  ']
    profile_dict = {}
    for j in range(len(profile_keys)):
        profile_dict[profile_keys[j]] = [profile_values[j]]
        
    profile_df = pd.DataFrame.from_dict(profile_dict, orient="columns")
    name = bio.split('/')[-1]
    name = name.replace('-', ' ')
    name = name.title()
    profile_df['Name'] = name
    profile_df_dict[name] = profile_df
    time.sleep(1)
    print(i, '/', len(links), name, 'Done.')

    

0 / 306 Conner Mantz Done.
1 / 306 Clayton Young Done.
2 / 306 Galen Rupp Done.
3 / 306 Sam Chelanga Done.
4 / 306 Scott Fauble Done.
5 / 306 Elkanah Kibet Done.
6 / 306 Zachery Panning Done.
7 / 306 Leonard Korir Done.
8 / 306 Futsum Zienasellassie Done.
9 / 306 Brian Shrader Done.
10 / 306 Matthew Alexander Mcdonald Done.
11 / 306 Nicolas Rodolfo Montanez Done.
12 / 306 Teshome Mekonen Done.
13 / 306 Clayton Albertson Done.
14 / 306 Joel William Reichow Done.
15 / 306 Reed Fischer Done.
16 / 306 Andrew Colley Done.
17 / 306 Kevin Salvano Done.
18 / 306 Josh Izewski Done.
19 / 306 Frank Lara Done.
20 / 306 Jacob Thomson Done.
21 / 306 Mick Iacofano Done.
22 / 306 Turner Wiley Done.
23 / 306 Tyler Pennel Done.
24 / 306 Kevin Lynch Done.
25 / 306 Jp Flavin Done.
26 / 306 Jonas Hampton Done.
27 / 306 Donnie Cowart Done.
28 / 306 Charlie Sweeney Done.
29 / 306 Kevin Lewis Done.
30 / 306 Reid Buchanan Done.
31 / 306 Jonny Phillips Done.
32 / 306 Colin Mickow Done.
33 / 306 Alan Peterson Do

272 / 306 Taylor Dare Done.
273 / 306 Amelia Keyser Gibson Done.
274 / 306 Kir Selert Done.
275 / 306 Laura Paulsen Done.
276 / 306 Johanna Butler Done.
277 / 306 Kidan Kidane Done.
278 / 306 Ann Marie Pierce Done.
279 / 306 Stephanie Rouse Done.
280 / 306 Madeline Elizabeth Duhon Done.
281 / 306 Lily Anderson Done.
282 / 306 Jennifer Pope Done.
283 / 306 Joanna Reyes Done.
284 / 306 Mary Denholm Done.
285 / 306 Alyssa Bloomquist Done.
286 / 306 Heather Kampf Done.
287 / 306 Kaitlin Donner Done.
288 / 306 Sylvia Bedford Done.
289 / 306 Lucy Dobbs Done.
290 / 306 Sofie Schunk Done.
291 / 306 Isabel Hebner Done.
292 / 306 Brittney Hall Done.
293 / 306 Theresa Hailey Done.
294 / 306 Julia Vasquez Giguere Done.
295 / 306 Kimberly Horner Done.
296 / 306 Carrie Mack Done.
297 / 306 Tara Welling Done.
298 / 306 Natalie Callister Done.
299 / 306 Stephanie Sherman Done.
300 / 306 Flannery Davis Love Done.
301 / 306 Adrian Walsh Done.
302 / 306 Caroline Williams Done.
303 / 306 Kayla Lampe Done.

In [163]:
column_list = []
for elem in profile_df_dict.values():
    col_list = list(elem.columns)
    column_list += col_list
column_list = list(set(column_list))
column_list

['Career Highlights',
 'Current Affiliation',
 'Favorite Song',
 'Number of attempts to qualify',
 'With sixty seconds’ worth of distance run,   \nYours is the Earth and everything that’s in it,   ',
 'Name',
 'Favorite Book',
 'Date of Birth',
 'Favorite TV Show/Series',
 'Age on Race Day',
 'Hometown',
 'Marathon PB',
 'Current Residence',
 'Previous Trials',
 'Favorite Movie',
 'College(s)',
 'Or walk with Kings — nor lose the common touch,\nIf neither foes nor loving friends can hurt you,',
 'Social Media',
 'First Trials qualification',
 'Interesting Story',
 'Favorite Post-race Indulgence',
 'High School',
 'Favorite Running Memory',
 'Favorite Breakfast',
 'Profession',
 'Favorite Motivational Quote',
 'Family',
 'Greatest Accomplishment']

In [164]:
len(column_list)

28

In [165]:
column_list = [col for col in column_list if len(col) < 50]
column_list

['Career Highlights',
 'Current Affiliation',
 'Favorite Song',
 'Number of attempts to qualify',
 'Name',
 'Favorite Book',
 'Date of Birth',
 'Favorite TV Show/Series',
 'Age on Race Day',
 'Hometown',
 'Marathon PB',
 'Current Residence',
 'Previous Trials',
 'Favorite Movie',
 'College(s)',
 'Social Media',
 'First Trials qualification',
 'Interesting Story',
 'Favorite Post-race Indulgence',
 'High School',
 'Favorite Running Memory',
 'Favorite Breakfast',
 'Profession',
 'Favorite Motivational Quote',
 'Family',
 'Greatest Accomplishment']

In [166]:
len(column_list)

26

In [167]:
for elem in profile_df_dict.keys():
    if  'With sixty seconds’ worth of distance run,   \nYours is the Earth and everything that’s in it,   ' in profile_df_dict[elem].columns:
        print(elem)

Veronica Eder


In [168]:
quote = """“If” by Rudyard Kipling - I’d recommend reading the entire poem, but here’s the last stanza: \n\nIf you can talk with crowds and keep your virtue,   ',
 'Or walk with Kings — nor lose the common touch,\nIf neither foes nor loving friends can hurt you,',
 'If all men count with you, but none too much;\nIf you can fill the unforgiving minute',
 'With sixty seconds’ worth of distance run,   \nYours is the Earth and everything that’s in it,   ',
 'And—which is more — you’ll be a Man, my son!"""

quote

"“If” by Rudyard Kipling - I’d recommend reading the entire poem, but here’s the last stanza: \n\nIf you can talk with crowds and keep your virtue,   ',\n 'Or walk with Kings — nor lose the common touch,\nIf neither foes nor loving friends can hurt you,',\n 'If all men count with you, but none too much;\nIf you can fill the unforgiving minute',\n 'With sixty seconds’ worth of distance run,   \nYours is the Earth and everything that’s in it,   ',\n 'And—which is more — you’ll be a Man, my son!"

In [169]:
profile_df_dict['Veronica Eder']['Favorite Motivational Quote'] = quote

In [172]:
elem = 'Conner Mantz'
for col in column_list:
    if col not in profile_df_dict[elem].columns:
        profile_df_dict[elem][col] = None
bad_col_list = [clm for clm in profile_df_dict[elem].columns if len(clm) > 50]
profile_df_dict[elem] = profile_df_dict[elem].drop(columns=bad_col_list)
profile_df_dict[elem] = profile_df_dict[elem].reindex(column_list, axis=1)
profile_df_dict[elem]

,Career Highlights,Current Affiliation,Favorite Song,Number of attempts to qualify,Name,Favorite Book,Date of Birth,Favorite TV Show/Series,Age on Race Day,Hometown,...,First Trials qualification,Interesting Story,Favorite Post-race Indulgence,High School,Favorite Running Memory,Favorite Breakfast,Profession,Favorite Motivational Quote,Family,Greatest Accomplishment
0,"6th, 2023 Bank of America Chicago Marathon; 20...",Nike,None,1,Conner Mantz,None,12/08/96,None,27,"Smithfield, UT",...,Yes,Studied mechanical engineering at BYU and work...,None,Sky View High School,None,None,None,None,He and wife Kylie were married a week after th...,None


In [173]:
for elem in profile_df_dict.keys():
    #print(elem)
    for col in column_list:
        if col not in profile_df_dict[elem].columns:
            profile_df_dict[elem][col] = None
    bad_col_list = [clm for clm in profile_df_dict[elem].columns if len(clm) > 50]
    profile_df_dict[elem] = profile_df_dict[elem].drop(columns=bad_col_list)
    profile_df_dict[elem] = profile_df_dict[elem].reindex(column_list, axis=1)

In [174]:
profile_df = pd.concat(profile_df_dict.values())
profile_df

,Career Highlights,Current Affiliation,Favorite Song,Number of attempts to qualify,Name,Favorite Book,Date of Birth,Favorite TV Show/Series,Age on Race Day,Hometown,...,First Trials qualification,Interesting Story,Favorite Post-race Indulgence,High School,Favorite Running Memory,Favorite Breakfast,Profession,Favorite Motivational Quote,Family,Greatest Accomplishment
0,"6th, 2023 Bank of America Chicago Marathon; 20...",Nike,None,1,Conner Mantz,None,12/08/96,None,27,"Smithfield, UT",...,Yes,Studied mechanical engineering at BYU and work...,None,Sky View High School,None,None,None,None,He and wife Kylie were married a week after th...,None
0,"4-time USATF Champion (8K, 10K, 15K and 20K); ...",ASICS,Shinjuku Speed Dating,1,Clayton Young,Atlas Shrugged,09/14/93,MrBeast,30,"American Fork, Utah",...,No,"I had knee surgery in February of 2023, then r...",Ice Cream,American Fork High School,Hitting the Olympic standard with my training ...,Ashley's Breakfast Sandwich,Mechanical Engineer at Stryker,"""Consistent Competence Equals Eventual Excelle...","My wife, Ashley, and I have been married for 7...",I'm a family man.
0,"2012 Olympic Silver Medalist 10,000m; 2016 Oly...",Nike,None,1,Galen Rupp,None,05/08/86,Seinfeld,37,"Portland, OR",...,No,I will be looking to qualify for my 5th Olympi...,Pizza,Central Catholic High School,None,Waffles,None,None,Married to Keara Rupp; they have three childre...,Husband and father
0,7-time USATF Champion; 2023 Pan Am Games Silve...,World Class Army Program,Dreams: Judah & The Lion and NEEDTOBREATHE,2,Sam Chelanga,The Bible,02/23/85,Ax Men,38,"Kabarsel, Kenya",...,No,I retired in 2018 to join the U.S. Army and th...,Burger or pizza,Bartolimo Boys High School,The time I got lost in my friend Jared Ward's ...,Bagel,U.S Army,"""It’s now or never""","Wife is Marybeth, an OTQ qualifier with half m...",Being a dad
0,Top American in the last three Boston Marathon...,Nike/Team Boss,None,None,Scott Fauble,None,11/05/91,None,32,"Golden, CO",...,No,After graduating from the University of Portla...,None,Wheat Ridge High School,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,"Winner, 2023 GoPro Mountain Games 20K; Winner,...",Tracksmith,Anything Bad Bunny,2,Adrian Walsh,To Kill a Mockingbird,09/14/94,Most recently - White Lotus,29,"Bozeman, MT",...,Yes,None,Cinco de Mayo breakfast burrito,Cate School,None,'gurt & granola (Beyond the Moon specifically!),Family Nurse Practitioner at Yampa Valley Medi...,"""The only thing we have to fear is fear itself...","My husband, Ben Kurtz, and I have been married...",I'm proud to have followed my career dreams an...
0,None,Green Racing Project,Don't Stop Me Now by Queen,2,Caroline Williams,None,07/17/92,None,31,"Westfield, NJ",...,Yes,My journey to the start line of the 2024 Trial...,Cheeseburger and an IPA,Mount Saint Mary Academy,Finding out I qualified for the Trials by 3 se...,"Taylor ham, egg, and cheese on an everything b...","Director of HR, Flagship Pioneering","""Do not spoil what you have by desiring what y...","My husband, Eddie Mulder, ran for Emory Univer...",My ability to balance a professional career an...
0,"18th, 2021 Bank of America Chicago Marathon; 2...",Philadelphia Runner Track Club,No music while running and then constantly lis...,4,Kayla Lampe,None,12/21/92,None,31,"Downingtown, PA",...,No,I qualified for the 2020 Trials but had to pul...,"Burger, fries and beer",Bishop Shanahan High School,When I finished the TCS New York City Marathon...,Egg and Cheese Bagel,Emergency Room RN - Baystate Franklin Medical ...,"""It is what it is.""",I have a Husky who runs 2-4 miles with me occa...,None
0,"7th, 2022 Medtronic Twin Cities Marathon; Winn...",None,The Middle by Jimmy Eat World,1,Hannah Becker,The Hiding Place by Corrie Ten Boom,10/15/98,Suits,25,"Eau Claire, WI",...,Yes,"In 9th grade, I decided to quit track. I was t...",Hamburgers,Eau Claire Memorial High School,Going into the D1 high 

In [186]:
profile_df = profile_df.reset_index(drop=True)

In [175]:
bad_names = [name for name in list(profile_df['Name']) if '%' in name]
bad_names

['Lyle O%27Brien',
 'Joseph %28Jp%29 P Trojan',
 'Sean O%E2%80%99Connor',
 'Keira D%27Amato',
 'Margareta %28Maggie%29 Montoya',
 'Jess %28Tonn%29 Mcclain',
 'Megan O%E2%80%99Neil']

In [176]:
for name in bad_names:
    for n in list(df['Name']):
        if name.split(' ')[0] == n.split(' ')[0]:
            print(n)

Lyle O'Brien
Joseph Whelan
Joseph Trojan
Sean O'Connor
Keira D'Amato
Megan O'Neil


In [178]:
for name in bad_names:
    for n in list(df['Name']):
        if name.split(' ')[-1] == n.split(' ')[-1]:
            print(n)

Joseph Trojan
Maggie Montoya


In [179]:
for n in list(df['Name']):
    if 'ain' in n:
        print(n)

Johnny Crain
Betsy Saina
Elaina Tabb
Jessica McClain
Elaine Estes


In [180]:
profile_df['Name'] = profile_df['Name'].replace('Lyle O%27Brien', "Lyle O'Brien")
profile_df['Name'] = profile_df['Name'].replace('Joseph %28Jp%29 P Trojan', "Joseph Trojan")
profile_df['Name'] = profile_df['Name'].replace('Sean O%E2%80%99Connor', "Sean O'Connor")
profile_df['Name'] = profile_df['Name'].replace("Keira D%27Amato", "Keira D'Amato")
profile_df['Name'] = profile_df['Name'].replace('Margareta %28Maggie%29 Montoya', "Maggie Montoya")
profile_df['Name'] = profile_df['Name'].replace('Jess %28Tonn%29 Mcclain', "Jessica McClain")
profile_df['Name'] = profile_df['Name'].replace('Megan O%E2%80%99Neil', "Megan O'Neil")

In [183]:
df['Lowercase Name'] = df['Name'].str.lower()
profile_df['Lowercase Name'] = profile_df['Name'].str.lower()

In [184]:
len(profile_df)

306

In [188]:
counter = 0
for elem in list(profile_df['Lowercase Name']):
    if elem in list(df['Lowercase Name']):
        counter +=1
    else:
        print(elem)
counter

matthew alexander mcdonald
nicolas rodolfo montanez
clayton albertson
joel william reichow
jonny phillips
jackson c neff
zach herriott
ben kendell
samuel doud
travis j morrison
aaron gruen
ben olson
t roy brown
garret lee
eric vazquez olivas
kellen w blumberg
colin buck
dominic g. hockenbury
ben payne
spencer l bossi johnson
alex taylor
daniel vassallo
jake heslington
jakob a kintzele
dylan j marx
dakotah marie lindwurm
jane bareikis
neely spence gracey
elena hayday
sydney t devore
molly roberts
anne marie blaney
sarah pagano buchanan
bridget lyons belyeu
mackenzie ann caldwell
olivia grace pratt
sara passani
kaylee a flanagan
libby bigelow
ari hendrix roach
amelia keyser gibson
kir selert
johanna butler
madeline elizabeth duhon
julia vasquez giguere


261

In [189]:
df.to_csv('qual_data.csv', index=False)
profile_df.to_csv('profile_data.csv', index=False)

In [187]:
profile_df.head()

,Career Highlights,Current Affiliation,Favorite Song,Number of attempts to qualify,Name,Favorite Book,Date of Birth,Favorite TV Show/Series,Age on Race Day,Hometown,...,Interesting Story,Favorite Post-race Indulgence,High School,Favorite Running Memory,Favorite Breakfast,Profession,Favorite Motivational Quote,Family,Greatest Accomplishment,Lowercase Name
0,"6th, 2023 Bank of America Chicago Marathon; 20...",Nike,None,1,Conner Mantz,None,12/08/96,None,27,"Smithfield, UT",...,Studied mechanical engineering at BYU and work...,None,Sky View High School,None,None,None,None,He and wife Kylie were married a week after th...,None,conner mantz
1,"4-time USATF Champion (8K, 10K, 15K and 20K); ...",ASICS,Shinjuku Speed Dating,1,Clayton Young,Atlas Shrugged,09/14/93,MrBeast,30,"American Fork, Utah",...,"I had knee surgery in February of 2023, then r...",Ice Cream,American Fork High School,Hitting the Olympic standard with my training ...,Ashley's Breakfast Sandwich,Mechanical Engineer at Stryker,"""Consistent Competence Equals Eventual Excelle...","My wife, Ashley, and I have been married for 7...",I'm a family man.,clayton young
2,"2012 Olympic Silver Medalist 10,000m; 2016 Oly...",Nike,None,1,Galen Rupp,None,05/08/86,Seinfeld,37,"Portland, OR",...,I will be looking to qualify for my 5th Olympi...,Pizza,Central Catholic High School,None,Waffles,None,None,Married to Keara Rupp; they have three childre...,Husband and father,galen rupp
3,7-time USATF Champion; 2023 Pan Am Games Silve...,World Class Army Program,Dreams: Judah & The Lion and NEEDTOBREATHE,2,Sam Chelanga,The Bible,02/23/85,Ax Men,38,"Kabarsel, Kenya",...,I retired in 2018 to join the U.S. Army and th...,Burger or pizza,Bartolimo Boys High School,The time I got lost in my friend Jared Ward's ...,Bagel,U.S Army,"""It’s now or never""","Wife is Marybeth, an OTQ qualifier with half m...",Being a dad,sam chelanga
4,Top American in the last three Boston Marathon...,Nike/Team Boss,None,None,Scott Fauble,None,11/05/91,None,32,"Golden, CO",...,After graduating from the University of Portla...,None,Wheat Ridge High School,None,None,None,None,None,None,scott fauble


In [138]:
bio = "https://www.orlando2024trials.com/athlete/414/34/veronica-eder"
r = requests.get(bio) 
soup = BeautifulSoup(r.content, 'html5lib')
soup

<!DOCTYPE html>
<html lang="en_US"><head>
<title>Veronica Eder - 2024 U.S. Olympic Team Trials Marathon</title>

<base href="https://www.orlando2024trials.com/"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>

<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link crossorigin="anonymous" href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0-alpha3/dist/css/bootstrap.min.css" integrity="sha384-KK94CHFLLe+nY2dmCWGMq91rCGa5gtU4mk92HdvYe+M/SXH301p5ILy+dN9+nJOZ" rel="stylesheet"/>
<link href="https://fonts.googleapis.com" rel="preconnect"/>
<link crossorigin="" href="https://fonts.gstatic.com" rel="preconnect"/>
<link href="https://fonts.googleapis.com/css2?family=Khand:wght@400;600;700&amp;family=Roboto:ital,wght@0,400;0,700;1,400;1,700&amp;display=swap" rel="stylesheet"/>
<link href="assets/fontawesome/css/all.min.css" rel="stylesheet"/>
<link href="https://cdnjs.cloudflare.com/ajax/libs/animate.css/4.1.1/animate.min.css" rel="stylesheet"/>
<link h

In [139]:
profile = soup.find_all(class_="profiles")[0]
profile.text

"\n    \n    Date of Birth\n    02/06/95\n    \n    \n    Age on Race Day\n    28\n    \n    \n    Current Residence\n    Philadelphia, PA\n    \n    \n    Hometown\n    Doylestown, PA\n    \n    \n    High School\n    Central Bucks East High School\n    \n    \n    Current Affiliation\n    Liberty Track Club\n    \n    \n    Profession\n    Senior Data Analyst at Vanguard\n    \n    \n    Marathon PB\n    2:35:48\n    \n    \n    \n    Social Media\n    \n\n    \n    \n    First Trials qualification\n    Yes\n    \n    \n    \n    Number of attempts to qualify\n    1\n    \n    \n    College(s)\n    Auburn University\n    \n    \n    Career Highlights\n    11th, 2018 USATF Cross Country Championships; 21st, 2021 Bank of America Chicago Marathon (16th American); 32nd, 2022 USATF Marathon Championships; 2024 Trials qualifier; 3-time NCAA Championships qualifier; 2-time All-American, 2-time Auburn University record-holder (3,000m indoor, 5,000m outdoor)\n    \n    \n    Family\n    I com

In [140]:
profile = profile.text.split('\n    ')
profile

['',
 '',
 'Date of Birth',
 '02/06/95',
 '',
 '',
 'Age on Race Day',
 '28',
 '',
 '',
 'Current Residence',
 'Philadelphia, PA',
 '',
 '',
 'Hometown',
 'Doylestown, PA',
 '',
 '',
 'High School',
 'Central Bucks East High School',
 '',
 '',
 'Current Affiliation',
 'Liberty Track Club',
 '',
 '',
 'Profession',
 'Senior Data Analyst at Vanguard',
 '',
 '',
 'Marathon PB',
 '2:35:48',
 '',
 '',
 '',
 'Social Media',
 '\n',
 '',
 '',
 'First Trials qualification',
 'Yes',
 '',
 '',
 '',
 'Number of attempts to qualify',
 '1',
 '',
 '',
 'College(s)',
 'Auburn University',
 '',
 '',
 'Career Highlights',
 '11th, 2018 USATF Cross Country Championships; 21st, 2021 Bank of America Chicago Marathon (16th American); 32nd, 2022 USATF Marathon Championships; 2024 Trials qualifier; 3-time NCAA Championships qualifier; 2-time All-American, 2-time Auburn University record-holder (3,000m indoor, 5,000m outdoor)',
 '',
 '',
 'Family',
 "I come from a big, close-knit family rooted in the Philadelph

In [141]:
profile = [elem for elem in profile if elem != '']
profile

['Date of Birth',
 '02/06/95',
 'Age on Race Day',
 '28',
 'Current Residence',
 'Philadelphia, PA',
 'Hometown',
 'Doylestown, PA',
 'High School',
 'Central Bucks East High School',
 'Current Affiliation',
 'Liberty Track Club',
 'Profession',
 'Senior Data Analyst at Vanguard',
 'Marathon PB',
 '2:35:48',
 'Social Media',
 '\n',
 'First Trials qualification',
 'Yes',
 'Number of attempts to qualify',
 '1',
 'College(s)',
 'Auburn University',
 'Career Highlights',
 '11th, 2018 USATF Cross Country Championships; 21st, 2021 Bank of America Chicago Marathon (16th American); 32nd, 2022 USATF Marathon Championships; 2024 Trials qualifier; 3-time NCAA Championships qualifier; 2-time All-American, 2-time Auburn University record-holder (3,000m indoor, 5,000m outdoor)',
 'Family',
 "I come from a big, close-knit family rooted in the Philadelphia area. My parents, who met in high school, decided to settle in Doylestown after growing up in Northeast Philly. I'm the oldest of three, with a bro

In [142]:
profile_keys = profile[0::2]
profile_keys

['Date of Birth',
 'Age on Race Day',
 'Current Residence',
 'Hometown',
 'High School',
 'Current Affiliation',
 'Profession',
 'Marathon PB',
 'Social Media',
 'First Trials qualification',
 'Number of attempts to qualify',
 'College(s)',
 'Career Highlights',
 'Family',
 'Interesting Story',
 'Favorite Book',
 'Favorite Movie',
 'Favorite TV Show/Series',
 'Favorite Song',
 'Favorite Post-race Indulgence',
 'Favorite Motivational Quote',
 'Or walk with Kings — nor lose the common touch,\nIf neither foes nor loving friends can hurt you,',
 'With sixty seconds’ worth of distance run,   \nYours is the Earth and everything that’s in it,   ',
 'Favorite Running Memory',
 '\n\n  ']

In [122]:
profile_values = profile[1::2]
profile_values

['10/12/91',
 '32',
 'Flagstaff, AZ/Providence, RI',
 'Chesterfield, MO',
 'Parkway Central High School',
 'New Balance',
 '2:18:29',
 '\n\n',
 'No',
 '2020',
 'Providence College',
 '-American record-holder, Marathon (2:18:29)\n-2020 Olympics, 10,000m, 10th\n-#2 all-time American, half marathon (1:06:52, Houston 2023)\n-2-time World Athletics Championships 10,000m (9th 2017; 10th 2019)\n-8-time USATF Champion (from 5K to half marathon)\n-2-time NCAA Champion, 5000m (indoors and outdoors, 2015) and NCAA record-holder, 5000m indoors (15:12.22)\n\nAfter a 10-time All-American career at Providence College, Sisson won the 2016 USATF 10 km Championships in her first full year as a pro, and then in 2017 ran the fastest half marathon debut ever by an American woman (1:08:21) when she finished as runner-up at the United Airlines NYC Half. In 2019, Sisson really came into her own: She kicked it off the with another great 13.1-miler, a 1:07:30 at the Aramco Houston Half Marathon (just five secon

In [123]:
profile_keys = [elem for elem in profile_keys if len(elem) > 0 and elem != '\n\n  ']
profile_keys

['Date of Birth',
 'Age on Race Day',
 'Current Residence',
 'Hometown',
 'High School',
 'Current Affiliation',
 'Marathon PB',
 'Social Media',
 'First Trials qualification',
 'Previous Trials',
 'College(s)',
 'Career Highlights',
 'Family']

In [124]:
profile_dict = {}
for i in range(len(profile_keys)):
    profile_dict[profile_keys[i]] = [profile_values[i]]
profile_dict

{'Date of Birth': ['10/12/91'],
 'Age on Race Day': ['32'],
 'Current Residence': ['Flagstaff, AZ/Providence, RI'],
 'Hometown': ['Chesterfield, MO'],
 'High School': ['Parkway Central High School'],
 'Current Affiliation': ['New Balance'],
 'Marathon PB': ['2:18:29'],
 'Social Media': ['\n\n'],
 'First Trials qualification': ['No'],
 'Previous Trials': ['2020'],
 'College(s)': ['Providence College'],
 'Career Highlights': ['-American record-holder, Marathon (2:18:29)\n-2020 Olympics, 10,000m, 10th\n-#2 all-time American, half marathon (1:06:52, Houston 2023)\n-2-time World Athletics Championships 10,000m (9th 2017; 10th 2019)\n-8-time USATF Champion (from 5K to half marathon)\n-2-time NCAA Champion, 5000m (indoors and outdoors, 2015) and NCAA record-holder, 5000m indoors (15:12.22)\n\nAfter a 10-time All-American career at Providence College, Sisson won the 2016 USATF 10 km Championships in her first full year as a pro, and then in 2017 ran the fastest half marathon debut ever by an A

In [125]:
profile_df = pd.DataFrame.from_dict(profile_dict, orient="columns")
profile_df

,Date of Birth,Age on Race Day,Current Residence,Hometown,High School,Current Affiliation,Marathon PB,Social Media,First Trials qualification,Previous Trials,College(s),Career Highlights,Family
0,10/12/91,32,"Flagstaff, AZ/Providence, RI","Chesterfield, MO",Parkway Central High School,New Balance,2:18:29,\n\n,No,2020,Providence College,"-American record-holder, Marathon (2:18:29)\n-...","In August 2018, Sisson married Shane Quinn, al..."


In [126]:
name = bio.split('/')[-1]
name = name.replace('-', ' ')
name = name.title()
profile_df['Name'] = name